# **Filogenia**

A filogenia é um campo que estuda as relações evolutivas entre organismos. Existem várias técnicas e métodos para a construção de árvores filogenéticas, e a escolha dependerá do tipo de dados disponíveis e das perguntas a responder.

Para análise filogenética utilizando o algoritmo **Needleman-Wunsch** devemos garantir estas etapas:

- **Loop de Alinhamento**:

    *Este loop deve invocar o algoritmo ***Needleman-Wunsch*** para cada par de sequências e, em seguida, atualizar a matriz ou estrutura de dados que armazena as pontuações ou distâncias entre as sequências.*

- **Matriz com um Dado Score**:

    *A matriz pode ser inicializada com zeros, e o loop de alinhamento será responsável por preenchê-la com os valores de pontuação calculados pelo algoritmo ***Needleman-Wunsch.***

- **Cálculo e Agrupamento Progressivo**:

    *Após realizar os alinhamentos progressivos e preencher a matriz de distâncias, é necessário implementar um método para calcular as distâncias progressivas e agrupar as sequências de acordo com essas distâncias. ***nw_ciclico***

- **Profit**:

    *O "Profit" refere-se à avaliação da qualidade do agrupamento ou à construção da árvore filogenética. Isso pode envolver a análise da estabilidade do agrupamento, a avaliação do bootstrap, ou a construção da árvore final.*

Para iniciar a **análise filogenética** necessitamos de fazer algumas modificações aos algoritmos desenvolvidos previamente apenas para o alinhamento:

In [7]:
def needleman_wunsch(seq1, seq2, score_subst, score_space):
    """
    Modificações/alterações feitas ao algoritmo base:
    - Adição de Pontuações de Espaço: As pontuações de espaço agora são aplicadas durante a inicialização da primeira coluna e linha da matriz de pontuação.
    - Aprimoramento no Traceback: Durante o traceback, o código agora verifica se o caminho escolhido é uma correspondência (D), exclusão (U), ou inserção (L). Isso é útil para gerar corretamente as sequências alinhadas.
    - Escolha da Melhor Opção: O código agora seleciona a melhor opção entre correspondência, exclusão e inserção, usando a função max e atribuindo best_choice à posição atual na matriz de pontuação.
    - Scores.
    """
    len_1, len_2 = len(seq1), len(seq2)
     
    # Inicializar a matriz de pontuação e a matriz de traceback
    score = [[0] * (len_2 + 1) for _ in range(len_1 + 1)]
    traceback = [[''] * (len_2 + 1) for _ in range(len_1 + 1)]
     
    # Inicializar a primeira coluna e linha com penalidades de espaço
    for i in range(len_1 + 1):
        score[i][0]     = i * score_space
        traceback[i][0] = 'D'
    for j in range(len_2 + 1):
        score[0][j]     = j * score_space
        traceback[0][j] = 'I'

    # Preencher a matriz de pontuação e a matriz de traceback
    for i in range(1, len_1 + 1):
        for j in range(1, len_2 + 1):
            match = score[i-1][j-1] + (score_subst if seq1[i-1] == seq2[j-1] else -score_subst)
            delete = score[i-1][j] + score_space
            insert = score[i][j-1] + score_space

            choices = [match, delete, insert]
            best_choice = max(choices)

            score[i][j] = best_choice
            traceback[i][j] = 'M' if best_choice == match else ('D' if best_choice == delete else 'I')

    # Realizar o traceback para obter as sequências alinhadas
    aligned_seq1, aligned_seq2 = '', ''
    i, j = len_1, len_2
    while i > 0 and j > 0:
        if traceback[i][j] == 'M':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif traceback[i][j] == 'D':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        elif traceback[i][j] == 'I':
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1

    return aligned_seq1, aligned_seq2, score[len_1][len_2]

# Modificações: Adição de Parâmetro de Comprimento (length): Agora a função aceita um comprimento desejado como parâmetro, garantindo que a sequência seja estendida até o comprimento especificado.
def aprimorar_seq(string_1, length):
    """
    Adiciona espaços (lacunas) ao final da sequência para atingir o comprimento desejado.
    """
    return string_1 + '-' * (length - len(string_1))


# Imprime a matriz de pontuação.
def print_matrix(string_1, string_2, matrix):
    """
    Modificações:
    - Adição de Rótulos de Sequência: As sequências string_1 e string_2 são agora usadas para rotular as linhas e colunas da matriz.
    """
    print(f'    {string_2}')
    for i, row in enumerate(matrix):
        if i < len(string_1): 
            row_str = ' '.join(map(str, row))
            print(f'{string_1[i]} {row_str}')
        else:
            break


## "def **nw_ciclico**"
Após adaptar os algoritmos dos alinhamentos progressivos e matriz de distâncias, é necessário implementar um método para calcular as distâncias progressivas e agrupar as sequências de acordo com essas distâncias: **nw_ciclico**

In [8]:
# A função tem como objetivo calcular a dissimilaridade entre todas as sequências em sequencias usando o algoritmo de Needleman-Wunsch
def nw_ciclico(sequencias, score_subst, score_space):
    """
    A função itera sobre todas as combinações únicas de pares de sequências, alinha cada par usando o algoritmo de Needleman-Wunsch e calcula a dissimilaridade com base no score obtido.
    A dissimilaridade é calculada por 1-(score/max(len(aligned_seq_i), len(aligned_seq_j)).
    A matriz de dissimilaridade resultante (distancia_matriz) é simétrica.
    """
    num_sequencias = len(sequencias)
    distancia_matriz = [[0.0] * num_sequencias for _ in range(num_sequencias)]

    for i in range(num_sequencias):
        for j in range(i + 1, num_sequencias):
            nome_i, seq_i = sequencias[i]
            nome_j, seq_j = sequencias[j]

            try:
                aligned_seq_i, aligned_seq_j, score = needleman_wunsch(seq_i, seq_j, score_subst, score_space)
                dissimilaridade        = 1.0 - (score / max(len(aligned_seq_i), len(aligned_seq_j)))
                distancia_matriz[i][j] = dissimilaridade
                distancia_matriz[j][i] = dissimilaridade
            except Exception as e:
                print(f"Failed to align sequence {nome_i} and {nome_j}: {e}")

    return distancia_matriz

## **Árvore Filogenética**

In [9]:
from Bio import pairwise2
from Bio import Phylo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from io import StringIO

def criar_arvore_filogenetica(distancia_matriz, sequencias):
    """
    This function uses the Biopython library for phylogenetic tree calculation and construction based on the provided sequences.
    """
    # Get the names and sequences of the entities
    nomes = [nome for nome, _ in sequencias]

    # Calculate the maximum length just once
    max_len = len(max(sequencias, key=lambda x: len(x[1]))[1])
    
    # Create an instance of MultipleSeqAlignment with the sequences adjusted to the same length
    msa = MultipleSeqAlignment([SeqRecord(Seq(aprimorar_seq(seq, max_len)), id=nome) for nome, seq in sequencias])

    # Calculate the distance matrix using the identity as a metric
    calculator = DistanceCalculator('identity')
    dm         = calculator.get_distance(msa)

    # Build the phylogenetic tree using the UPGMA method
    constructor = DistanceTreeConstructor()
    tree        = constructor.upgma(dm)

    # Print the phylogenetic tree
    print(tree)



A partir dos algoritmos modificados, nw_ciclico, e uma função para "desenhar" a árvore a partir da distância da matriz podemos criar dois tipo de árvores filogenéticas em python.

### Exemplo:

In [10]:

# Exemplo de uso com 2 seqs
s1 = 'ATCTCGGCTAAAC'
s2 = 'ACCGTATCTCGGTGCA'

# Lista de tuplos contendo nome e sequência, alterar estes parâmetros para o número de sequências a analisar
sequencias = [('s1', s1), ('s2', s2)]

# Parâmetros da substituição ou da adição de espaços, mais parametros poderão ser incorporados no código e escolhidos aqui
score_subst = 2
score_space = -4

# Aplicar Needleman-Wunsch ciclico
distancia_matriz = nw_ciclico(sequencias, score_subst, score_space)

# Cria a 1ª árvore filogenética 
print("1ª árvore:")
criar_arvore_filogenetica(distancia_matriz, sequencias)

# Hardcoded sequence names replaced by actual sequences' names in the second tree generation
tree_str = "(({}:0.375, {}:0.375)Inner1:0.0)RootedTree;".format(sequencias[0][0], sequencias[1][0])

# Cria a 2ª árvore filogenética usando outro método
print("2ª árvore:")
tree = Phylo.read(StringIO(tree_str), "newick")

# Exibir a árvore em formato de texto
Phylo.draw_ascii(tree)

1ª árvore:
Tree(rooted=True)
    Clade(branch_length=0, name='Inner1')
        Clade(branch_length=0.375, name='s2')
        Clade(branch_length=0.375, name='s1')
2ª árvore:
  ___________________________________________________________________________ s1
_|
 |___________________________________________________________________________ s2



Os testes unitários vão verificar se todas as funções utilizadas neste capítulo estão a funcionar conforme o esperado.

In [ ]:
import unittest

class TestSequenciaTools(unittest.TestCase):
    def test_needleman_wunsch(self):
        """
        O test_needleman_wunsch verifica se a saída da função needleman_wunsch corresponde às sequências alinhadas esperadas. Seria bom adicionar uma asserção para o score também, se tiver sido previamente calculada.
        """
        seq1 = 'ATCTCGGCTAAAC'
        seq2 = 'ACCGTATCTCGGTGCA'
        pass

        # Configura sistema de pontuação
        # Para uma correspondência (substituição), será atribuída uma pontuação de 2
        score_subst = 2
        # Para um gap (espaço), será deduzida uma pontuação de -4
        score_space = -4

        # Chama a função Needleman-Wunsch e obtém as sequências alinhadas retornadas e a pontuação
        aligned_seq1, aligned_seq2, score = needleman_wunsch(seq1, seq2, score_subst, score_space)

        # Testa os resultados conforme o esperado
        self.assertEqual(aligned_seq1, 'ATCTCGGCTAAAC---')
        self.assertEqual(aligned_seq2, '----ACCGTATCTCG')
        self.assertEqual(score, 4)

    def test_aprimorar_seq(self):
        """
        A função aprimorar_seq estende uma sequência dada para um certo comprimento, adicionando '-' no final
        """
        string = 'ATCTCGGCTAAAC'
        length = 20
        result = aprimorar_seq(string, length)
        
        # Teste de resultado
        self.assertEqual(result, 'ATCTCGGCTAAAC-------')

    def test_nw_ciclico(self):
        """
        O teste unitário test_nw_ciclico apenas verifica se nw_ciclico é executado sem erro. Seria muito mais robusto se pudesse acrescentar uma asserção para verificar a matriz de distância resultante contra uma saída correta conhecida.
        """
        s1 = 'ATCTCGGCTAAAC'
        s2 = 'ACCGTATCTCGGTGCA'
        sequencias = [('s1', s1), ('s2', s2)]
        score_subst = 2
        score_space = -4
        distancia_matriz = nw_ciclico(sequencias, score_subst, score_space)
        
        # Verifica se a matriz de distância é simétrico
        for i in range(len(distancia_matriz)):
            for j in range(i + 1, len(distancia_matriz[i])):
                self.assertEqual(distancia_matriz[i][j], distancia_matriz[j][i])
        pass

    def test_criar_arvore_filogenetica(self):
        """
        O teste unitário test_criar_arvore_filogenetica verifica se criar_arvore_filogenetica é executado sem erro para as entradas fornecidas. Da mesma forma que o test_nw_ciclico, incluir uma asserção aqui para verificar a saída iria reforçar este teste. No entanto, verificar a saída exata da função criar_arvore_filogenetica pode ser difícil, pois é provável que esteja a gerar uma representação em string da árvore.
        """
        s1 = 'ATCTCGGCTAAAC'
        s2 = 'ACCGTATCTCGGTGCA'
        sequencias = [('s1', s1), ('s2', s2)]
        score_subst = 2
        score_space = -4
        distancia_matriz = nw_ciclico(sequencias, score_subst, score_space)
        
        # Chama a função para criar a árvore filogenética
        criar_arvore_filogenetica(distancia_matriz, sequencias)
        pass

if __name__ == '__main__':
    unittest.main()